## What is Semantic Kernel?

In [4]:
!pip install semantic-kernel

The main thing about semantic-kernel is the "kernel" --> this is the brain and orchestrator of all the LLMs, plugins and memory.

In [ ]:
from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.functions import KernelArguments
from dotenv import load_dotenv
import os

load_dotenv()

# Create a kernel
kernel = Kernel()

# Connect to OpenAI
kernel.add_service(OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini", 
    api_key=os.getenv("OPENAI_API_KEY")
))

PROMPT = """
You are concise.
Give me the TL;DR of this in exactly {{$words}} words:

{{$text}}
"""

args = KernelArguments(words=4, text="Semantic Kernel helps you wire LLMs to your own tools and memory.")
result = await kernel.invoke_prompt(PROMPT, arguments=args)
print(result)

Integrates LLMs with tools.


In [18]:
import asyncio
from typing import Annotated
from semantic_kernel import Kernel
from semantic_kernel.functions import kernel_function, KernelArguments
from semantic_kernel.functions.kernel_plugin import KernelPlugin
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion

class MathPlugin:
    @kernel_function(description="Multiply two integers.")
    def multiply(
        self,
        a: Annotated[int, "first number"],
        b: Annotated[int, "second number"],
    ) -> int:
        return a * b

async def main():
    kernel = Kernel(services=OpenAIChatCompletion(
        ai_model_id="gpt-4o-mini", 
        api_key=os.getenv("OPENAI_API_KEY")
    ))
    # Wrap your object as a plugin and add it to the kernel
    math = KernelPlugin.from_object("Math", MathPlugin())
    kernel.plugins["Math"] = math

    # Call your tool directly
    out = await kernel.invoke(function_name="multiply", plugin_name="Math",
                              arguments=KernelArguments(a=6, b=7))
    print(out)

await main()


42


In [22]:
from semantic_kernel.agents import ChatCompletionAgent
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion

agent = ChatCompletionAgent(
    name="Helper",
    instructions="Use tools when helpful. Be concise.",
    service=OpenAIChatCompletion(
        ai_model_id="gpt-4o-mini", 
        api_key=os.getenv("OPENAI_API_KEY")
    ),
    plugins=[MathPlugin()],  # classes with @kernel_function methods
)

async def main():
    reply = await agent.get_response("What is 13 * 37?")
    print(reply.content)  # Agent may call MathPlugin.multiply under the hood

await main()


13 * 37 = 481.
